# Postupci praćenja objekata u računalnom vidu radnim okvirom OpenCV
  
 ### Diplomski projekt: http://www.fer.unizg.hr/predmet/pro_dipl, siječanj 2018. FER, 2017/2018
 #### Autor: Tomislav Lokotar
 #### Mentor: izv. prof. dr. sc. Zoran Kalafatić

# 1. Uvod

OpenCV je vrlo raširen radni okvir koji nudi mnoge mogućnosti u kontekstu računalnog vida. Praćenje objekata u pokretu čest je problem kojeg je moguće upotrijebiti u razne svrhe. U ovom radu iznesen je kratak pregled implementiranih metoda za praćenje objekata, te njihova usporedba na javno dostupnim snimkama. Aktualna verzija u trenutku pisanja je OpenCV 3.4.0 u kojoj su implementirani slijedeći algoritmi praćenja: MedianFlow, MIL, GOTURN, KCF, Boosting, MOSSE i TLD. U sklopu projekta razvijen je i softver za usporedbu ugrađenih metoda praćenja. Skup podataka na kojem su algoritmi testirani je TB-50 [1]. TB-50 je skup 50 označenih sekvenci na kojima je označen najmanje jedan objekt koji se prati. Svi algoritmi, osim GOTURN s greškom u implementaciji [2], su uspješno testirani. Na slici 1 prikazan je prozor generiram skriptom za testiranje praćenja na videu s pokretnom kamerom, u procesu praćenja košarkaškog igrača algoritmom MIL.

![slika1](docs/kosarka.png)
<center>Slika 1: praćenje unaprijed označenog košarkaša na videosnimci</center> 


In [29]:
import cv2
import numpy as np
import cv_utils
import bench_utils
import tb_50
import benchmark
import sys
import platform

In [31]:
cv_utils.info()

v = cv2.__version__.split(".")
cv_ver = tuple([int(e) for e in v])

if sys.version_info < (3, 5, 0):
    RuntimeError("Potrebna je novija verzija Python3 programskom jezika")

if cv_ver < (3, 3, 0):
    RuntimeError("Potrebna je novije verzija OpenCV programskog paketa")
    
print("\nPlatforma na kojoj je skripta pokrenuta:\n")

print(platform.machine())
print(platform.version())
print(platform.platform())
print(platform.uname())
print(platform.system())
print(platform.processor())

Python version: 3.5.2
OpenCV version: 3.4.0

Available tracking algorithms:
TrackerKCF, TrackerMOSSE, TrackerGOTURN, TrackerMedianFlow, TrackerBoosting, TrackerMIL, TrackerTLD

Platforma na kojoj je skripta pokrenuta:

x86_64
#35~16.04.1-Ubuntu SMP Thu Jan 25 10:13:43 UTC 2018
Linux-4.13.0-32-generic-x86_64-with-LinuxMint-18.3-sylvia
uname_result(system='Linux', node='Z570', release='4.13.0-32-generic', version='#35~16.04.1-Ubuntu SMP Thu Jan 25 10:13:43 UTC 2018', machine='x86_64', processor='x86_64')
Linux
x86_64


In [3]:
algorithms = cv_utils.tracking_algorithms()

for alg in algorithms:
    print(alg)

TrackerKCF
TrackerMOSSE
TrackerGOTURN
TrackerMedianFlow
TrackerBoosting
TrackerMIL
TrackerTLD


U rječniku <i>algorithms</i> pohranjeni su uređeni parovi konstruktora objekata koji predstavljaju algoritme praćenja, te pripadajuće funkcije za njihovo instanciranje.

In [4]:
print(algorithms['TrackerGOTURN'])

(<class 'cv2.TrackerGOTURN'>, <built-in function TrackerGOTURN_create>)


In [5]:
goturn_create = algorithms['TrackerGOTURN'][1]

goturn_tracker = goturn_create()

U trenutnoj verziji OpenCV-a nije moguće pratiti objekt algoritmom GOTURN [3], pa je njegova funkcionalnost izuzeta iz evaluacije algoritama.

# 2 Analiza uspješnosti algoritama praćenja

## 2.1 Mjera uspješnosti praćenja

Kao mjera uspješnosti praćenja objekata korištena je funkcija $$C_{suc}(f)= \frac{\left | r\cap  r_g \right |}{\left | r \cup r_g\right |  }$$ na sličan način kao što su je koristili Janku, Koplik, Dulik[4]. Varijabla <i>r</i> predstavlja pravokutnik označen algoritmom praćenja, dok je <i>r<sub>g</sub></i> pravokutnik ručno označen na korištenim sekvencama. Idealna vrijednost, kad algoritam praćenja označi pravokutnik koji se poklapa s označenim, je 1. 0 je najmanja vrijednost i predstavlja promašaj u postupku praćenja. Mjerenje se vrši za svaku sličicu sekvence. Postupak testiranja počinje inicijalizacijom algoritma praćenja s ručno označenim pravokutnikom. Ukoliko algoritam praćenja u nekom trenutku postigne vrijednost funkcije <i>C<sub>suc</sub></i> ispod postavljene granice (obično 0.5 ili više), algoritam se reinicijalizira na ručno označeno područje, a broj reinicijalizacija se inkrementira. U situaciji praćenja objekta u stvarnom vremenu, reinicijalizacija se može poistovjetiti s upotrijebom detektora objekata.

## 2.2 Podaci za testiranje

Skup podataka na kojima su testirani algoritmi je TB-50[5]. TB-50 je javno dostupan označen skup podataka sa sekvencama u svrhu evalucije algoritama praćenja. Svaka sekvenca ima postavljenu jednu ili više zastavica ovisno o vrsti problema. Vrste problema uključuju nagle promjene osvjetljenja na objekt u praćenju, nagle promjene udaljenosti objekata, prekrivanje objekta, deformacije, zamućenost, brze pokrete, rotacije, napuštanje okvira snimke, tešku razlučivost objekta u odnosu na njegovu pozadinu, te snimke niske rezolucije.

Popis video datoteka:

In [6]:
benchmark_sequences = tb_50.get_all_videos_and_groundtruths()

for directory, video_path, ground_truth_file in benchmark_sequences:
    print(directory, end=" ")
print()

./TB-50/Basketball/ ./TB-50/Biker/ ./TB-50/Bird1/ ./TB-50/BlurBody/ ./TB-50/BlurCar2/ ./TB-50/BlurFace/ ./TB-50/BlurOwl/ ./TB-50/Box/ ./TB-50/Car1/ ./TB-50/Car4/ ./TB-50/CarDark/ ./TB-50/CarScale/ ./TB-50/ClifBar/ ./TB-50/Couple/ ./TB-50/Crowds/ ./TB-50/Deer/ ./TB-50/DragonBaby/ ./TB-50/Dudek/ ./TB-50/Girl/ ./TB-50/Human6/ ./TB-50/Human9/ ./TB-50/Ironman/ ./TB-50/Jump/ ./TB-50/Jumping/ ./TB-50/Matrix/ ./TB-50/MotorRolling/ ./TB-50/Panda/ ./TB-50/RedTeam/ ./TB-50/Shaking/ ./TB-50/Singer2/ ./TB-50/Skating1/ ./TB-50/Skiing/ ./TB-50/Soccer/ ./TB-50/Surfer/ ./TB-50/Sylvester/ ./TB-50/Tiger2/ ./TB-50/Trellis/ ./TB-50/Walking/ ./TB-50/Walking2/ ./TB-50/Woman/ 


Primjer pokretanja funkcije za evaluaciju za algoritam MedianFlow, video Basketball. Prikazani su dostupne informacije o izvođenju algoritma.

In [9]:
result = benchmark.bench(algorithms['TrackerMedianFlow'][1], 'TB-50/Basketball/video.avi', 
                'TB-50/Basketball/groundtruth_rect.txt', reset_iou = 0.5, name='MedianFlow primjer')

print(result.keys())
print()

dict_keys(['tracker', 'video', 'fps', 'avg_iou', 'reset_count', 'bboxes', 'ious', 'iou_reset'])



Neke informacije o izvođenju:

In [10]:
for arg in ('avg_iou', 'fps', 'reset_count'):
    print(arg, ":", result[arg])

avg_iou : 0.7423716824547375
fps : 127.091595318266
reset_count : 24


Pokušaj pokretanja za algoritam GOTURN[6]:

In [15]:
try:
    result = benchmark.bench(algorithms['TrackerGOTURN'][1], 'TB-50/Basketball/video.avi', 
                    'TB-50/Basketball/groundtruth_rect.txt', reset_iou = 0.5, name='GOTURN primjer')
except:
    print("pogreska!")
    
# za detalje o pogrešci potrebno je odkomentirati slijedeću liniju koda:
# result = benchmark.bench(algorithms['TrackerGOTURN'][1], 'TB-50/Basketball/video.avi', 
                    # 'TB-50/Basketball/groundtruth_rect.txt', reset_iou = 0.5, name='GOTURN primjer')

pogreska!


Kako u nastavku ne bi bilo problema s algoritmom GOTURN, izbacuje se iz rječnika dostupnih algoritama: 

In [19]:
# Za kasnije verzije OpenCV-a (3.4.x novije od 3.4.0) s očekivanom ispravkom pogreške, 
# potrebno je obrisati ili zakomentirati slijedece linije, kako bi se testiranje obavilo i za algoritam GOTURN:

if "TrackerGOTURN" in algorithms:
    del algorithms["TrackerGOTURN"]

TB-50 dataset je označen atributima ovisno o problemima koje predstavlja za algoritme praćenja. Definirani su slijedeći problemi:

In [33]:
IV = "Illumination Variation"
SV = "Scale Variation"
OCC = "Occlusion"
DEF = "Deformation"
MB = "Motion Blur"
FM = "Fast Motion"
IPR = "In-plane Rotation"
OPR = "Out-of-Plane Rotation"
OV = "Out-of-View"
BC = "Background Clutters"
LR = "Low Resolution"

problem_types = dict()

Svaka sekvenca iz TB-50 ima dodijeljen jedan ili više problema:

In [39]:
problem_types[IV] = ["Basketball", "Box", "Car1", "Car2", "Car24", "Car4", "CarDark", "Coke", "Crowds", "David", "Doll", "FaceOcc2", "Fish", "Human2", "Human4.2", "Human7", "Human8", "Human9", "Ironman", "KiteSurf", "Lemming", "Liquor", "Man", "Matrix", "Mhyang", "MotorRolling", "Shaking", "Singer1", "Singer2", "Skating1", "Skiing", "Soccer", "Sylvester", "Tiger1", "Tiger2", "Trans", "Trellis", "Woman"]
problem_types[SV] = ["Biker", "BlurBody", "BlurCar2", "BlurOwl", "Board", "Box", "Boy", "Car1", "Car24", "Car4", "CarScale", "ClifBar", "Couple", "Crossing", "Dancer", "David", "Diving", "Dog", "Dog1", "Doll", "DragonBaby", "Dudek", "FleetFace", "Freeman1", "Freeman3", "Freeman4", "Girl", "Girl2", "Gym", "Human2", "Human3", "Human4.2", "Human5", "Human6", "Human7", "Human8", "Human9", "Ironman", "Jump", "Lemming", "Liquor", "Matrix", "MotorRolling", "Panda", "RedTeam", "Rubik", "Shaking", "Singer1", "Skater", "Skater2", "Skating1", "Skating2.1", "Skating2.2", "Skiing", "Soccer", "Surfer", "Toy", "Trans", "Trellis", "Twinnings", "Vase", "Walking", "Walking2", "Woman"]
problem_types[OCC] = ["Basketball", "Biker", "Bird2", "Bolt", "Box", "CarScale", "ClifBar", "Coke", "Coupon", "David", "David3", "Doll", "DragonBaby", "Dudek", "FaceOcc1", "FaceOcc2", "Football", "Freeman4", "Girl", "Girl2", "Human3", "Human4.2", "Human5", "Human6", "Human7", "Ironman", "Jogging.1", "Jogging.2", "Jump", "KiteSurf", "Lemming", "Liquor", "Matrix", "Panda", "RedTeam", "Rubik", "Singer1", "Skating1", "Skating2.1", "Skating2.2", "Soccer", "Subway", "Suv", "Tiger1", "Tiger2", "Trans", "Walking", "Walking2", "Woman"]
problem_types[DEF] = ["Basketball", "Bird1", "Bird2", "BlurBody", "Bolt", "Bolt2", "Couple", "Crossing", "Crowds", "Dancer", "Dancer2", "David", "David3", "Diving", "Dog", "Dudek", "FleetFace", "Girl2", "Gym", "Human3", "Human4.2", "Human5", "Human6", "Human7", "Human8", "Human9", "Jogging.1", "Jogging.2", "Jump", "Mhyang", "Panda", "Singer2", "Skater", "Skater2", "Skating1", "Skating2.1", "Skating2.2", "Skiing", "Subway", "Tiger1", "Tiger2", "Trans", "Walking", "Woman"]
problem_types[MB] = ["Biker", "BlurBody", "BlurCar1", "BlurCar2", "BlurCar3", "BlurCar4", "BlurFace", "BlurOwl", "Board", "Box", "Boy", "ClifBar", "David", "Deer", "DragonBaby", "FleetFace", "Girl2", "Human2", "Human7", "Human9", "Ironman", "Jump", "Jumping", "Liquor", "MotorRolling", "Soccer", "Tiger1", "Tiger2", "Woman"]
problem_types[FM] = ["Biker", "Bird1", "Bird2", "BlurBody", "BlurCar1", "BlurCar2", "BlurCar3", "BlurCar4", "BlurFace", "BlurOwl", "Board", "Boy", "CarScale", "ClifBar", "Coke", "Couple", "Deer", "DragonBaby", "Dudek", "FleetFace", "Human6", "Human7", "Human9", "Ironman", "Jumping", "Lemming", "Liquor", "Matrix", "MotorRolling", "Skater2", "Skating2.1", "Skating2.2", "Soccer", "Surfer", "Tiger1", "Tiger2", "Toy", "Vase", "Woman"]
problem_types[IPR] = ["Bird2", "BlurBody", "BlurFace", "BlurOwl", "Bolt", "Box", "Boy", "CarScale", "ClifBar", "Coke", "Dancer", "David", "David2", "Deer", "Diving", "Dog1", "Doll", "DragonBaby", "Dudek", "FaceOcc2", "FleetFace", "Football", "Football1", "Freeman1", "Freeman3", "Freeman4", "Girl", "Gym", "Ironman", "Jump", "KiteSurf", "Matrix", "MotorRolling", "MountainBike", "Panda", "RedTeam", "Rubik", "Shaking", "Singer2", "Skater", "Skater2", "Skiing", "Soccer", "Surfer", "Suv", "Sylvester", "Tiger1", "Tiger2", "Toy", "Trellis", "Vase"]
problem_types[OPR] = ["Basketball", "Biker", "Bird2", "Board", "Bolt", "Box", "Boy", "CarScale", "Coke", "Couple", "Dancer", "David", "David2", "David3", "Dog", "Dog1", "Doll", "DragonBaby", "Dudek", "FaceOcc2", "FleetFace", "Football", "Football1", "Freeman1", "Freeman3", "Freeman4", "Girl", "Girl2", "Gym", "Human2", "Human3", "Human6", "Ironman", "Jogging.1", "Jogging.2", "Jump", "KiteSurf", "Lemming", "Liquor", "Matrix", "Mhyang", "MountainBike", "Panda", "RedTeam", "Rubik", "Shaking", "Singer1", "Singer2", "Skater", "Skater2", "Skating1", "Skating2.1", "Skating2.2", "Skiing", "Soccer", "Surfer", "Sylvester", "Tiger1", "Tiger2", "Toy", "Trellis", "Twinnings", "Woman"]
problem_types[OV] = ["Biker", "Bird1", "Board", "Box", "ClifBar", "DragonBaby", "Dudek", "Human6", "Ironman", "Lemming", "Liquor", "Panda", "Suv", "Tiger2"]
problem_types[BC] = ["Basketball", "Board", "Bolt2", "Box", "Car1", "Car2", "Car24", "CarDark", "ClifBar", "Couple", "Coupon", "Crossing", "Crowds", "David3", "Deer", "Dudek", "Football", "Football1", "Human3", "Ironman", "Liquor", "Matrix", "Mhyang", "MotorRolling", "MountainBike", "Shaking", "Singer2", "Skating1", "Soccer", "Subway", "Trellis"]
problem_types[LR] = ["Biker", "Car1", "Freeman3", "Freeman4", "Panda", "RedTeam", "Skiing", "Surfer", "Walking"]

# 3. Rezultati

U nastavku su prikazani rezultati uspješnosti izvođenja pojedinih algoritama:

In [40]:
# parameters:

IOU_RESET_VALUES = [0.5, 0.6, 0.7]

In [ ]:
# initialization:

results = dict()

# load from hdd


In [ ]:
# benchmark process:

for iou_res in IOU_RESET_VALUES:
    for directory, video_path, ground_truth_file in benchmark_sequences:
        for algorithm_name in sorted(algorithms.keys()):
            if not (iou_res, directory, algorithm_name) in results:
                results[(iou_res, directory, algorithm_name)] = benchmark.bench(algorithms[algorithm_name][1], 
                                                                                video_path, 
                                                                                ground_truth_file, 
                                                                                reset_iou = iou_res,
                                                                                name=algorithm_name)